# 🧬 GenBank / EMBL Sequence Parser with GC Analysis

In this project, we:
- Parse multiple GenBank (.gb, .gbk) or EMBL (.embl) files.
- Extract `gene` and `CDS` features: start, end, strand, product, translation.
- Compute GC content for CDS regions.
- Flag interesting features:
  - Long genes (>1000 bp)
  - High GC content (>60%)
- Explore data in an interactive table.
- Save results to a CSV for further analysis.

Powered by **BioPython + pandas + Google Colab widgets**.


In [1]:
!pip install biopython qgrid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 889.2/889.2 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.9 MB/s eta 0:00:00
  Created wheel for qgrid: filename=qgrid-1.3.1-py2.py3-none-any.whl size=1761252 sha256=976880481aebe0ea49da0a5e36458fcf90aac0b30b002a9079f9a772f4cd40da
  Stored in directory: /root/.cache/pip/wheels/7f/0f/d6/33b60aa35dbdc3d3e84c44b780c85e92d8767b698843f9e256
Successfully built qgrid


In [10]:
from Bio import SeqIO
from Bio.SeqUtils import gc_fraction
import pandas as pd
import qgrid
from google.colab import files
from google.colab import data_table

##📂  Upload your GenBank or EMBL files

In [ ]:
uploaded = files.upload()
filenames = list(uploaded.keys())
print(f"Uploaded files: {filenames}")

##🧬  Parse & analyze sequences

In [4]:
df = pd.DataFrame(columns=[
    'File', 'Record_ID', 'Type', 'Start', 'End', 'Length', 'Strand',
    'Gene', 'Product', 'Protein Translation', 'GC Content',
    'Long Gene', 'High GC'
])

for file in filenames:
    if file.endswith(('.gb', '.gbk', '.embl')):
        format_type = "genbank" if file.endswith(('.gb', '.gbk')) else "embl"

        for record in SeqIO.parse(file, format_type):
            for feature in record.features:
                if feature.type in ["gene", "CDS"]:
                    start = int(feature.location.start)
                    end = int(feature.location.end)
                    strand = feature.location.strand
                    length = end - start

                    gene_name = feature.qualifiers.get("gene", ["-"])[0]
                    product = feature.qualifiers.get("product", ["-"])[0]
                    translation = feature.qualifiers.get("translation", ["-"])[0] if "translation" in feature.qualifiers else "-"

                    gc_content = 0
                    if feature.type == "CDS":
                        seq = feature.extract(record.seq)
                        gc_content = round(gc_fraction(seq) * 100, 2)

                    long_gene = length > 1000
                    high_gc = gc_content > 60

                    df = pd.concat([df, pd.DataFrame([{
                        'File': file,
                        'Record_ID': record.id,
                        'Type': feature.type,
                        'Start': start,
                        'End': end,
                        'Length': length,
                        'Strand': strand,
                        'Gene': gene_name,
                        'Product': product,
                        'Protein Translation': translation,
                        'GC Content': gc_content,
                        'Long Gene': long_gene,
                        'High GC': high_gc
                    }])], ignore_index=True)

## 🔍 Explore the parsed features
Use interactive tables to filter and sort by:
- GC Content
- Length
- Product
- Gene name
and more!


In [5]:
data_table.DataTable(df, num_rows_per_page=10)

,File,Record_ID,Type,Start,End,Length,Strand,Gene,Product,Protein Translation,GC Content,Long Gene,High GC


In [6]:
qgrid_widget = qgrid.show_grid(df, show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [7]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

## 💾 Save the data to CSV
Download the processed data to analyze later in Excel, R, Python etc.


In [9]:
df.to_csv("genbank_embl_parsed_with_gc_flags.csv", index=False)
files.download("genbank_embl_parsed_with_gc_flags.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ✅ Summary
- Parsed `gene` & `CDS` features from multiple GenBank / EMBL files.
- Computed **GC content** for each CDS.
- Flagged long genes (>1000 bp) & high GC (>60%).
- Explored data with interactive filters.
- Saved everything to a tidy CSV.

---

🚀 Now use this for:
- Genome annotation summaries
- Comparing multiple organisms
- Reporting in research projects
